# Exercise 11-1, 11-3 & 11-4

### DSC530
#### Taniya Adhikari
##### 11/05/2020


In [508]:
from __future__ import print_function, division

%matplotlib inline

import numpy as np
import pandas as pd

import random

import thinkstats2
import thinkplot
import statsmodels.formula.api as smf

## Exercise 11-1

Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [371]:
import first

live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]

To choose the significant variables, I decided to run the functions GoMining for each variable and choose variables in order by choosing highest significant R-values for variables before pregnancy to lowest and re-running the MiningReport iteratively to narrow the list down. 

In [466]:
# searches variable that predict pregnancy lengths.
# returns: list of (rsquared, variable name) pairs
def GoMining(df):
    
    # list of variables
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            # formula without any explanatory variable
            formula = 'prglngth ~ +' + name
            
            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError, patsy.PatsyError):
            continue
        
        variables.append((results.rsquared, name))

    return variables

In [467]:
variables = GoMining(live)

In [468]:
# fetches description of each vvariable names and creates a dictionary.
def ReadVariables():

    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars

In [469]:
# Prints variables with R-squared between 0.1 to 1.0.
def MiningReport1(variables, n=30):
    
    all_vars = ReadVariables()  # calls function for description of variables

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            if 0.1 <= r2 <= 1:
                print(name + ': {}     '.format(r2) + desc)
        except (KeyError, IndexError):
            print(name + ': {}'.format(r2))

In [470]:
MiningReport1(variables)

prglngth: 1.0     DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest: 0.8062434116139248     GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
totalwgt_lb: 0.12445743148120225
birthwgt_lb: 0.11977307804917248     BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1: 0.10372542204583246     LOW BIRTHWEIGHT - BABY 1


**Looking at the list above, these variables do have high R-squared but are factors after pregnancy or the same as pregnancy length, so they are un-usable variables**

In [471]:
# Prints variables with R-squared below 0.005 but above 0.001.
def MiningReport2(variables, n=30):
    
    all_vars = ReadVariables()  # calls function for description of variables

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            if 0.005 <= r2 <= 0.1:
                print(name + ': {}     '.format(r2) + desc)
        except (KeyError, IndexError):
            if 0.005 <= r2 <= 0.1:
                print(name + ': {}'.format(r2))
            else:
                None

In [472]:
MiningReport2(variables)

mosgest: 0.09562431989592235     GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
prglngth_i: 0.022053775796472053     PRGLNGTH IMPUTATION FLAG


**Above list are also unusable variables for same reasons listed above**

In [473]:
# Prints variables with R-squared below 0.0005 but above 0.005.
def MiningReport3(variables, n=30):
    
    all_vars = ReadVariables()  # calls function for description of variables

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            if 0.001 <= r2 <= 0.005:
                print(name + ': {}     '.format(r2) + desc)
        except (KeyError, IndexError):
            if 0.0005 <= r2 < 0.005:
                print(name + ': {}'.format(r2))
            else:
                None


In [474]:
MiningReport3(variables)

nbrnaliv: 0.004577565785538473     BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
anynurse: 0.002452024883713211     BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
bfeedwks: 0.002369183944671338     DURATION OF BREASTFEEDING IN WEEKS
pregend1: 0.002249389433796045     BC-1 HOW PREGNANCY ENDED - 1ST MENTION
cmlastlb: 0.0020431424422018285     CM FOR R'S MOST RECENT LIVE BIRTH
fmarcon5_i: 0.001968159324254537     FMARCON5 IMPUTATION FLAG
evuseint: 0.0018917527758633979     EG-1 USE ANY METHOD IN PREGNANCY INTERVAL?
gestasun_m: 0.001657131955017932     BC-5 GESTATIONAL LENGTH OF PREGNANCY IN MONTHS
sest: 0.0013223681981657798     SCRAMBLED VERSION OF THE STRATUM
matchfound: 0.001309107377125751     CHECK ON WHETHER CHILD MATCHES BIO CHILD IN HH ROSTER - 1ST
cmlstprg: 0.0012828619646414463     CM FOR R'S MOST RECENT COMPLETED PREGNANCY
birthord: 0.001237273673660888     BIRTH ORDER
frsteatd_p: 0.0011558077327729066     BH-3 UNITS (MOS/WKS/DAYS) FOR FRSTEATD_N - 1ST F

Looking at the above list Here are the observations:  
nbrnaliv: is usable variable with highest R-squared = 0.004577565785538473.  
  
Running function MiningReport and GoMining again with 1st explanatory variables nbrnaliv. nbrnaliv is a variable of number of babies born alive in this pregnancy, we'll take all the values above 1 for this model. 


In [475]:
def GoMining1(df):
    
    # list of variables
    variables1 = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            # formula without any explanatory variable
            formula = 'prglngth ~ nbrnaliv>1 +' + name
            
            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError, patsy.PatsyError):
            continue
        
        variables1.append((results.rsquared, name))

    return variables1

In [480]:
# Prints variables with R-squared  with 1 explanatory variable.
def MiningReport4(variables, n=30):
    
    all_vars = ReadVariables()  # calls function for description of variables

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name + ': {}     '.format(r2) + desc)
        except (KeyError, IndexError):
            print(name + ': {}'.format(r2))

In [481]:
variables1 = GoMining1(live)

In [482]:
MiningReport4(variables1)

prglngth: 1.0     DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest: 0.8065254803450603     GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
totalwgt_lb: 0.12712542095891
birthwgt_lb: 0.1226876300027917     BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1: 0.10660760072636011     LOW BIRTHWEIGHT - BABY 1
mosgest: 0.10335149052021764     GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
prglngth_i: 0.03144282961028477     PRGLNGTH IMPUTATION FLAG
anynurse: 0.013555226239402196     BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
whentell: 0.012831472175839909     EG-20 WHEN DID R TELL FATHER OF PREG ABOUT PREGNANCY: DURING OR AFTER?
frsteatd_p: 0.012723783625517737     BH-3 UNITS (MOS/WKS/DAYS) FOR FRSTEATD_N - 1ST FROM THIS PREG
timokhp: 0.012434939471870021     EG-17 R BECAME PREG SOONER, RIGHT TIME, OR LATER THAN FATHER OF PREG WANTED
matchfound: 0.012132592344732296     CHECK ON WHETHER CHILD MATCHES BIO CHILD IN HH ROSTER - 1ST
frsteatd_n: 0.0119

Every variable seems like a variable after birth according to description. Except:  
wthpart1: 0.005720791728371921     EG-12A RIGHT BEFORE PREG, WANT TO HAVE BABY WITH THAT PARTNER?
birthord: 0.005574952040113268     BIRTH ORDER  
poverty: 0.005543390447730112     POVERTY LEVEL INCOME    

wthpart1 has lot of missing value. So I will skip that and move to birthord and poverty

In [483]:
model1 = smf.ols('prglngth ~ nbrnaliv>1 + birthord==1 + poverty', data=live)
results1 = model1.fit()
results1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     32.70
Date:                Mon, 09 Nov 2020   Prob (F-statistic):           5.16e-21
Time:                        02:09:19   Log-Likelihood:                -18250.
No. Observations:                8884   AIC:                         3.651e+04
Df Residuals:                    8880   BIC:                         3.654e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.7833      0.038   1009.260      0.000      38.708      38.859
nbrnaliv > 1[T.True]     -1.4804      0.165     -8.995      0.000      -1.803      -1.158
birthord == 1[T.True]     0.0926      0.041      2.284      0.022       0.013       0.172
poverty                   0.0003      0.000      2.506      0.012     7.6e-05       0.001
==============================================================================
Omnibus:                     1585.005   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6131.395
Skew:                          -0.851   Prob(JB):                         0.00
Kurtosis:                       6.697   Cond. No.                     2.09e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.09e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

These variable seems to be statistically significant with really low p-values but R-square is really low 0.011. It doesn't seem like it's going to help with the bet

## Exercise 11-3

**Exercise:** If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called `poisson`. It works the same way as `ols` and `logit`. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called `numbabes`.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [499]:
import nsfg
import patsy

resp = nsfg.ReadFemResp()
resp.index = resp.caseid
# joins the two data set and adds suffix r to the variable with same name
join = live.join(resp, on='caseid', rsuffix='_r')

# replaces null values to 100
join.numbabes.replace([100], np.nan, inplace=True)

In [507]:
# following code finds the variable names with description. We need age, education, race and income

all_vars = ReadVariables()  # calls function for description of variables
variables_names = [] 

for name in join.columns:
    try:
        if join[name].var() < 1e-7:
            continue
            
    except (ValueError, TypeError, patsy.PatsyError):
        continue
        
    variables_names.append((name))

variables_names.sort(reverse=True)
for name in variables_names:
    key = re.sub('_r$', '', name)
    try:
        desc = all_vars.loc[key].desc
        
        # check for series
        if isinstance(desc, pd.Series):
            
            # converts to string
            desc = desc.to_string()
            if ' education ' in desc.lower():
                print(name + ': ' + desc)
            elif 'income ' in desc.lower():
                print(name + ': ' + desc)
            elif 'race' in name.lower():
                print(name + ': ' + desc)
            else:
                None
       
        elif 'education ' in desc.lower():
            print(name + ': ' + desc)       
        elif 'income ' in desc.lower():
                print(name + ': ' + desc)        
        elif 'race' in name.lower():
            print(name + ': ' + desc)
        else:
            None
    
    except (KeyError, IndexError):
        print(name)


wage: JI-1A IN 2001 RECEIVED INCOME FROM WAGES/SALARIES
unemp: JI-1G IN 2001 RECEIVED INCOME FROM UNEMPLOYMENT COMPENSATION
totincr: TOTAL INCOME OF R'S FAMILY
totinc: JI-3 TOTAL COMBINED FAMILY INCOME IN 2001
totalwgt_lb
toincwmy: JI-2 PREFER TO REPORT TOTAL INCOME PER WEEK/MONTH/YEAR
ssi: JI-1F IN 2001 RECEIVED INCOME FROM SUPPLEMENTAL SECURITY
socsec: JI-1C IN 2001 RECEIVED INCOME FROM SOCIAL SECURITY
selfinc: JI-1B IN 2001 RECEIVED INCOME FROM SELF-EMPLOYMENT
secu_r
rscreenrace: R S RACE AS REPORTED IN SCREENER
retire: JI-1E IN 2001 RECEIVED INCOME FROM RETIREMENT
racehx7: CB-9 RACE OF 2ND HUSBAND - 2ND MENTION
racehx6: CB-9 RACE OF 2ND HUSBAND - 1ST MENTION
racehx4: CB-9 RACE OF 1ST HUSBAND - 4TH MENTION
racehx3: CB-9 RACE OF 1ST HUSBAND - 3RD MENTION
racehx2: CB-9 RACE OF 1ST HUSBAND - 2ND MENTION
racehx16: CB-9 RACE OF 4TH HUSBAND - 1ST MENTION
racehx11: CB-9 RACE OF 3RD HUSBAND - 1ST MENTION
racehx1: CB-9 RACE OF 1ST HUSBAND - 1ST MENTION
racecx2: CD-10 RACE OF 1ST FORMER COHAB

In [501]:
# poisson regression
# totincr: TOTAL INCOME OF R'S FAMILY
# educat: name
# educat    EDUCATION (COMPLETED YEARS OF SCHOOLING)
# age_r respondant's age
# race    RACE OF RESPONDENT

formula_poisson='numbabes ~ age_r + C(race) + totincr + educat'
model2 = smf.poisson(formula_poisson, data=join)
results2 = model2.fit()
results2.summary() 

Optimization terminated successfully.
         Current function value: 1.687055
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8878
Method:                           MLE   Df Model:                            5
Date:                Mon, 09 Nov 2020   Pseudo R-squ.:                 0.03109
Time:                        02:23:09   Log-Likelihood:                -14988.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                1.106e-205
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.0842      0.045     23.995      0.000       0.996       1.173
C(race)[T.2]    -0.1398      0.015     -9.464      0.000      -0.169      -0.111
C(race)[T.3]    -0.0914      0.025     -3.717      0.000      -0.140      -0.043
age_r            0.0208      0.001     20.474      0.000       0.019       0.023
totincr         -0.0179      0.002     -9.442      0.000      -0.022      -0.014
educat          -0.0443      0.003    -15.139      0.000      -0.050      -0.039
================================================================================
"""

Now we can predict the number of children for a woman who is 35 years old, black, and a college
graduate whose annual household income exceeds $75,000

In [502]:
# predictions
columns = ['age_r', 'race', 'totincr', 'educat']
new = pd.DataFrame([[35, 1, 14, 16]], columns=columns)
results2.predict(new)

0    2.342182
dtype: float64

**Between 2-3 children for the given demographics**

## Exercise 11-4

**Exercise:** If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called `mnlogit`. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called `rmarital`.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [498]:
# multinomial logistic regression
formula_mnlogit='rmarital ~ age_r + C(race) + totincr + educat'
model3 = smf.mnlogit(formula_mnlogit, data=join)
results3 = model3.fit()
results3.summary() 

Optimization terminated successfully.
         Current function value: 1.087603
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8854
Method:                           MLE   Df Model:                           25
Date:                Mon, 09 Nov 2020   Pseudo R-squ.:                  0.1655
Time:                        02:22:24   Log-Likelihood:                -9662.3
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.4532      0.279     15.977      0.000       3.907       5.000
C(race)[T.2]    -0.9219      0.089    -10.409      0.000      -1.095      -0.748
C(race)[T.3]    -0.6334      0.136     -4.674      0.000      -0.899      -0.368
age_r           -0.0570      0.006     -9.754      0.000      -0.068      -0.046
totincr         -0.1302      0.012    -11.298      0.000      -0.153      -0.108
educat          -0.2051      0.019    -11.017      0.000      -0.242      -0.169
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -4.5432      0.916     -4.960      0.000      -6.338      -2.748
C(race)[T.2]    -0.4405      0.236     -1.865      0.062      -0.904       0.023
C(race)[T.3]     0.0329      0.335      0.098      0.922      -0.623       0.689
age_r            0.1301      0.019      6.833      0.000       0.093       0.167
totincr         -0.3251      0.032    -10.182      0.000      -0.388      -0.263
educat          -0.1010      0.048     -2.100      0.036      -0.195      -0.007
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.6485      0.302     -8.783      0.000      -3.239      -2.057
C(race)[T.2]    -0.3188      0.093     -3.417      0.001      -0.502      -0.136
C(race)[T.3]    -0.7664      0.171     -4.485      0.000      -1.101      -0.431
age_r            0.0641      0.007      9.817      0.000       0.051       0.077
totincr         -0.2270      0.012    -19.588      0.000      -0.250      -0.204
educat           0.0673      0.017      4.036      0.000       0.035       0.100
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.8231      0.345      2.385      0.017       0.147       1.500
C(race)[T.2]    -1.0491      0.104    -10.125      0.000      -1.252      -0.846
C(race)[T.3]    -0.5644      0.156     -3.624      0.000      -0.870      -0.259
age_r            0.0063      0.007      0.872      0.383      -0.008       0.020
totincr         -0.2923      0.015    -20.149      0.000      -0.321      -0.264
educat          -0.0136      0.021     -0.639      0.523      -0.056       0.028
--------------------------------------------------------------------------------
  rmarital=6       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        6.2370      0.275     22.654      0.000       5.697       6.777
C

Make a prediction for a woman who is 25 years old, white, and a high
school graduate whose annual household income is about $45,000.

In [503]:
# predictions
columns = ['age_r', 'race', 'totincr', 'educat']
new = pd.DataFrame([[25, 2, 11, 12]], columns=columns)
results3.predict(new)

,0,1,2,3,4,5
0,0.748384,0.125474,0.001103,0.035295,0.023813,0.065931


This person has 74.8% chance of being married, 12.5% chance of cohabiting, 0.1% chance of widowed, 3.5% chance of separated and 6.5% chance of being never married